# **Importing Required Libraries**

In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer

# **Step 1 : Data Loading**

In [50]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
df = pd.read_csv(url, header=None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain',
           'capital-loss', 'hours-per-week', 'native-country', 'income']
, na_values=' ?', skipinitialspace=True, delimiter=',')

In [51]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# **Step 2 : Initial Data Inspection and Cleaning**

### Step 2.1 : Display dataset information and preview data

**Preview Data:** Using head() lets students see the actual records, which is critical for understanding the context of the data.

**Dataset Info:** The info() function shows non-null counts and datatypes, which helps in quickly spotting missing values or incorrect data formats.



In [52]:
# Display dataset information and preview data
# -> Display the first 5 rows and print the basic states for data
# -> Hint: head and info

print(df.head())
print(df.info())




   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country income  
0          2174             0              40  United-States  <=50K  
1             0             0             

**Dataset Dimensions:** Knowing the shape of the data informs students about its scale, which can affect computation time and choice of algorithms.

In [53]:
# Display the dimension of the data
# -> Hint : shape

print(df.shape)

(32561, 15)


### **Step 2.2 : Basic Statistical Summary:**

The describe() function provides vital statistics (min, max, mean, standard deviation) that help identify any outliers or anomalies in numerical columns.

In [54]:
# Check the min , max , count , means , standard diviation etc
# Hint -> describe

df.describe()



,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


# **Step 2.3 : Counting Missing Values:**

This step is essential for diagnosing data quality. Missing values can lead to biased or inaccurate models if not handled properly.

In [55]:
# Count missing values in each column

df.isnull().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education-num,0
marital-status,0
occupation,0
relationship,0
race,0
sex,0


### **Step 2.4 : Checking for Duplicate Records:**

Duplicates can skew the analysis by over-representing some data, so it's important to remove them.

In [56]:
# Check for duplicate records

print('Duplicate Count =', df.duplicated().sum())

Duplicate Count = 24


### **Step 2.5 : Inspecting Unique Values in Categorical Columns:**

Unique value inspection reveals if there are any unexpected values (e.g., a '?' or extra spaces) that need cleaning. This is important for ensuring reliable encoding later.

In [57]:
# Check the unique data of each Categorical column to find if there is any irrelevant record or data e.g one record contains ? mark

categorialcolumn = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']

for col in categorialcolumn:
    print(col)
    print(df[col].unique())
    print(f"Count of unique values: {df[col].nunique()}")

workclass
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
Count of unique values: 9
education
['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
Count of unique values: 16
marital-status
['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
Count of unique values: 7
occupation
['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
Count of unique values: 15
relationship
['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']
Count of unique values: 6
race
['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
Count of uniq

### **Step 2.6 : Validating and Converting Data Types:**

Converting columns to the correct datatype (like converting an ID column to a string) prevents errors in operations such as merging, filtering, or encoding.

In [58]:
# Check the data type of each column and if wrong datatype convert it to the suitable datatype

print("Data Types:")
print(df.dtypes)

Data Types:
age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object


### **Step 2.7 : Checking Value Counts for Categorical Columns:**

Value counts help in understanding the distribution within each category. They are useful for detecting class imbalances and anomalies.

In [59]:
# Check value count for each Cateorical column

for col in categorialcolumn:
    # print(col)
    print(f"Value in: {col}")
    print(df[col].value_counts())
    print(f"\nCount of unique values: {df[col].nunique()}")

Value in: workclass
workclass
Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

Count of unique values: 9
Value in: education
education
HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: count, dtype: int64

Count of unique values: 16
Value in: marital-status
marital-status
Married-civ-spouse       14976
Never-married            10683
Divorced                  4443
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse    

### **Step 2.8 : Handling Missing Values using SimpleImputer:**

Imputation preserves the dataset size while ensuring that no null values interfere with analysis. Different strategies are used for numerical (mean) and categorical (mode) columns based on their characteristics.

In [60]:
# TODO: fill null values either by mean , median  or mode based on type of data

# Separate numeric and categorical columns
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Numeric: fill with mean

numeric_imputer = SimpleImputer(strategy='mean')
df[numeric_columns] = numeric_imputer.fit_transform(df[numeric_columns])


# Categorical: fill with most frequent (mode)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_columns] = categorical_imputer.fit_transform(df[categorical_columns])


# **Step 3 : Converting Data Types and Cleaning Categorical Data**

### **Step 3.1 :Removing Leading/Trailing Spaces:**

Standardize entries in categorical columns so that no extra spaces lead to misclassification of similar values.



In [61]:
# Check for leading/trailing spaces in categorical data and remove them

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

for col in categorical_columns:
    df[col] = df[col].str.strip()

### **Step 3.2 : Checking and Converting Data Types:**

Ensure that every column is of the correct data type (e.g., IDs as strings, dates as datetime).

In [62]:
# Check the data type of each column and if wrong datatype convert it to the suitable datatype

df['income'] = df['income'].astype('category')

### **3.3 : Converting to 'category' Datatype:**

Transform columns that represent categorical data (like Gender or Embarked port) into the 'category' type to optimize memory and computational performance.

In [63]:
# Convert suitable columns to 'category' datatype

ategorical_columns = ['workclass', 'education', 'marital-status', 'occupation',
                       'relationship', 'race', 'sex', 'native-country', 'income']

for col in categorical_columns:
    df[col] = df[col].astype('category')

# Display updated data types
print("Updated data types:")
print(df.dtypes)

Updated data types:
age                float64
workclass         category
fnlwgt             float64
education         category
education-num      float64
marital-status    category
occupation        category
relationship      category
race              category
sex               category
capital-gain       float64
capital-loss       float64
hours-per-week     float64
native-country    category
income            category
dtype: object


# **Step 4 : Feature Engineering**

### **Step 4.1 : Creating "age_group" Feature:**

Binning converts continuous age values into meaningful categories (e.g., 'Young', 'Adult') that are easier to analyze and interpret.

In [64]:
# create feature "age_group" age categories using binning
bins = [0, 25, 45, 65, np.inf]
labels = ['Young', 'Adult', 'Middle-Aged', 'Senior']

# Create age groups using binning
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

# Display the first few rows with the new age_group column
print(df[['age', 'age_group']].head())

    age    age_group
0  39.0        Adult
1  50.0  Middle-Aged
2  38.0        Adult
3  53.0  Middle-Aged
4  28.0        Adult


### **Step 4.2 : Creating "education_hours_interaction" Feature:**

Interaction features help capture complex relationships between variables. In this case, the interaction between education (via 'education-num') and work intensity ('hours-per-week') may reveal underlying patterns related to social or economic outcomes.

In [65]:
# Create an interaction feature "education_hours_interaction": education-num multiplied by hours-per-week (as a proxy for workload vs. education level)

df['education_hours_interaction'] = df['education-num'] * df['hours-per-week']

# Display the first few rows with the new feature
print(df[['education-num', 'hours-per-week', 'education_hours_interaction']].head())

   education-num  hours-per-week  education_hours_interaction
0           13.0            40.0                        520.0
1           13.0            13.0                        169.0
2            9.0            40.0                        360.0
3            7.0            40.0                        280.0
4           13.0            40.0                        520.0


# **Step 5 : Encoding Categorical Data**

**One-Hot Encoding:** Converts multiple categorical values into binary columns to prevent ordinality.


In [66]:
# One-hot encode the categorical columns (sex, workclass, education, etc.).
# Select categorical columns for one-hot encoding
categorical_cols_to_encode = ['workclass', 'education', 'marital-status', 'occupation',
                              'relationship', 'race', 'sex', 'native-country', 'age_group']

# One-hot encode the categorical columns
encoded_df = pd.get_dummies(df, columns=categorical_cols_to_encode, drop_first=True)

# Display the shape after one-hot encoding
print("Shape after one-hot encoding:", encoded_df.shape)
print(encoded_df.head())

# Use label encoding for the income column, converting ≤50K to 0 and >50K to 1
le = LabelEncoder()
encoded_df['income_encoded'] = le.fit_transform(df['income'])

# Display the first few rows with the encoded income column
print(encoded_df[['income_encoded']].head())


Shape after one-hot encoding: (32561, 105)
    age    fnlwgt  education-num  capital-gain  capital-loss  hours-per-week  \
0  39.0   77516.0           13.0        2174.0           0.0            40.0   
1  50.0   83311.0           13.0           0.0           0.0            13.0   
2  38.0  215646.0            9.0           0.0           0.0            40.0   
3  53.0  234721.0            7.0           0.0           0.0            40.0   
4  28.0  338409.0           13.0           0.0           0.0            40.0   

  income  education_hours_interaction  workclass_Federal-gov  \
0  <=50K                        520.0                  False   
1  <=50K                        169.0                  False   
2  <=50K                        360.0                  False   
3  <=50K                        280.0                  False   
4  <=50K                        520.0                  False   

   workclass_Local-gov  ...  native-country_South  native-country_Taiwan  \
0              


**Label Encoding for Income:** Maps income to binary labels for binary classification tasks.

In [67]:
# Use label encoding for the income column, converting ≤50K to 0 and >50K to 1
le = LabelEncoder()
encoded_df['income_encoded'] = le.fit_transform(df['income'])

# Display the first few rows with the encoded income column
print(encoded_df[['income_encoded']].head())

   income_encoded
0               0
1               0
2               0
3               0
4               0


# **Step 6 : Normalization and Standardization**

Standardization transforms the specified columns to a mean of 0 and a standard deviation of 1, which is important to ensure comparability among numerical features during model training.

In [68]:
# Step 6: Normalization and Standardization
# Standardize the "age", "hours-per-week", "capital-gain" and "capital-loss" column

# Columns to standardize
cols_to_standardize = ['age', 'hours-per-week', 'capital-gain', 'capital-loss']

# Standardize the columns
for col in cols_to_standardize:
    mean = encoded_df[col].mean()
    std = encoded_df[col].std()
    encoded_df[col + '_standardized'] = (encoded_df[col] - mean) / std

# Display the first few rows with the standardized columns
print(encoded_df[[col + '_standardized' for col in cols_to_standardize]].head())

   age_standardized  hours-per-week_standardized  capital-gain_standardized  \
0          0.030670                    -0.035429                   0.148451   
1          0.837096                    -2.222119                  -0.145918   
2         -0.042641                    -0.035429                  -0.145918   
3          1.057031                    -0.035429                  -0.145918   
4         -0.775756                    -0.035429                  -0.145918   

   capital-loss_standardized  
0                  -0.216656  
1                  -0.216656  
2                  -0.216656  
3                  -0.216656  
4                  -0.216656  
